In [1]:
import numpy as np
from tensorflow import keras
from keras import layers

In [2]:
# Path to the images
path_to_images = "training-data/dataset1/known_images"

In [3]:
train_dataset, val_dataset = keras.utils.image_dataset_from_directory(
    path_to_images,
    class_names=["normal", "sick"],
    batch_size = 32,
    image_size=(395, 488),
    validation_split=0.2,
    subset="both",
    seed=200
)

Found 715 files belonging to 2 classes.
Using 572 files for training.
Using 143 files for validation.


In [4]:
# Instantiating and freezing the VGG19 convolutional base
conv_base = keras.applications.vgg19.VGG19(
    weights="imagenet",
    include_top=False,
)

conv_base.trainable = False

80134624/80134624 [==============================] - 561s 7us/step


In [5]:
# Adding a data augmentation stage and a classifier to the convolutional base
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2)
    ]
)

In [6]:
inputs = keras.Input(shape=(395, 488, 3))
x = data_augmentation(inputs)

x = keras.applications.vgg19.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

In [7]:
model.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"]
)

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="chest-opacities-detection-with-VGG19-pretrained-model-V2.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

In [ ]:
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=callbacks
)

Epoch 1/50


2023-05-17 07:40:15.463006: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


18/18 [==============================] - 524s 30s/step - loss: 111.7683 - accuracy: 0.7675 - val_loss: 6.5421 - val_accuracy: 0.9441
Epoch 2/50
18/18 [==============================] - 302s 16s/step - loss: 33.3472 - accuracy: 0.8497 - val_loss: 7.7472 - val_accuracy: 0.9161
Epoch 3/50
18/18 [==============================] - 273s 15s/step - loss: 10.6036 - accuracy: 0.9545 - val_loss: 5.6198 - val_accuracy: 0.9371
Epoch 4/50
18/18 [==============================] - 268s 15s/step - loss: 19.6883 - accuracy: 0.9091 - val_loss: 13.7385 - val_accuracy: 0.9231
Epoch 5/50
18/18 [==============================] - 288s 16s/step - loss: 5.4896 - accuracy: 0.9598 - val_loss: 4.0227 - val_accuracy: 0.9580
Epoch 6/50
18/18 [==============================] - 505s 28s/step - loss: 10.9549 - accuracy: 0.9441 - val_loss: 32.1740 - val_accuracy: 0.7832
Epoch 7/50
18/18 [==============================] - 508s 29s/step - loss: 11.9983 - accuracy: 0.9318 - val_loss: 1.8780 - val_accuracy: 0.9790
Epoch 8/

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)

# Accuracy plots
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()

# Loss plots
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()